In [1]:
#export
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random
import numpy as np

In [2]:
start = datetime.now()
X = list()
y = list()
XX = list()
with open("pima-indians-diabetes.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in set([i for i in range(0, 9)]):
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

In [3]:
def entropy(class_y):

    entropy = 0
    ### Implement your code here
    #############################################
    class_y = np.array(class_y)
    (unique, counts) = np.unique(class_y, return_counts=True)
    p = np.array(counts)/len(class_y)
    entropy = round(np.sum(-1*p.T*np.log2(p)), 3)
    #############################################
    return entropy


In [4]:
def partition_classes(X, y, split_attribute, split_val):
    # Inputs:
    #   X               : data containing all attributes
    #   y               : labels
    #   split_attribute : column index of the attribute to split on
    #   split_val       : a numerical value to divide the split_attribute

    # TODO: Partition the data(X) and labels(y) based on the split value - BINARY SPLIT.
    # 
    # Split_val should be a numerical value
    # For example, your split_val could be the mean of the values of split_attribute
    #
    # You can perform the partition in the following way
    # Numeric Split Attribute:
    #   Split the data X into two lists(X_left and X_right) where the first list has all
    #   the rows where the split attribute is less than or equal to the split value, and the 
    #   second list has all the rows where the split attribute is greater than the split 
    #   value. Also create two lists(y_left and y_right) with the corresponding y labels.

    '''
    Example:

    X = [[3, 10],                 y = [1,
         [1, 22],                      1,
         [2, 28],                      0,
         [5, 32],                      0,
         [4, 32]]                      1]

    Here, columns 0 and 1 represent numeric attributes.

    Consider the case where we call the function with split_attribute = 0 and split_val = 3 (mean of column 0)
    Then we divide X into two lists - X_left, where column 0 is <= 3  and X_right, where column 0 is > 3.


    X_left = [[3, 10],                 y_left = [1,
              [1, 22],                           1,
              [2, 28]]                           0]

    X_right = [[5, 32],                y_right = [0,
               [4, 32]]                           1]

    ''' 

    X_left = []
    X_right = []

    y_left = []
    y_right = []
    ### Implement your code here
    #############################################
    X = np.array(X)
    y = np.array(y)
    data = np.concatenate((X,y.reshape(-1, 1)), axis=1)
    left = data[data[:, split_attribute] <= split_val]
    left1 = np.hsplit(left, len(X[0])+1)
    right = data[data[:, split_attribute] > split_val]
    right1 = np.hsplit(right, len(X[0])+1)

    X_left = np.concatenate((left1[0:len(X[0])]), axis=1).tolist()
    X_right = np.concatenate((right1[0:len(X[0])]), axis=1).tolist()
    y_left = left1[-1].astype(int).tolist()
    y_right = right1[-1].astype(int).tolist()

    #############################################
    return (X_left, X_right, y_left, y_right)



In [5]:
def information_gain(previous_y, current_y):
    info_gain = 0
    ### Implement your code here
    #############################################
    entropy_list = []
    for i in range(len(current_y)):
        current_y[i] = np.array(current_y[i])
        (u, c) = np.unique(current_y[i], return_counts=True)
        p = len(current_y[i])/len(previous_y)
        entropy_list.append(p * entropy(current_y[i]))
    entropy_sum = np.sum(entropy_list)
    info_gain = entropy(previous_y) - entropy_sum

    #############################################
    return info_gain


In [6]:
def best_split(X, y):
    # Inputs:
    #   X       : Data containing all attributes
    #   y       : labels
    #   TODO    : For each node find the best split criteria and return the split attribute, 
    #             spliting value along with  X_left, X_right, y_left, y_right (using partition_classes) 
    #             in the dictionary format {'split_attribute':split_attribute, 'split_val':split_val, 
    #             'X_left':X_left, 'X_right':X_right, 'y_left':y_left, 'y_right':y_right, 'info_gain':info_gain}

    split_attribute = 0
    split_val = 0
    X_left, X_right, y_left, y_right = [], [], [], []
    ### Implement your code here
    #############################################
    y_array = np.array(y)
    X_array = np.hsplit(np.array(X), len(X[0]))

    ig_val = 0 
    best_val = 0
    best_attr = 0
    for attr in range(0,len(X[0])):
        for val in X_array[attr]:
            X_l, X_r, y_l, y_r = partition_classes(X, y, attr, val)
            ig = information_gain(y, [y_l, y_r])
            if ig > ig_val:
                ig_val = ig
                best_val = val
                best_attr = attr

    X_left, X_right, y_left, y_right = partition_classes(X, y, best_attr, best_val)


    return {'split_attribute':best_attr, 'split_val':best_val, \
         'X_left':X_left, 'X_right':X_right, 'y_left':y_left, 'y_right':y_right, 'info_gain':ig_val}


In [7]:
tree = {}
max_depth = 10

def learn(X, y, par_node = {}, depth=0):
    # TODO: Train the decision tree (self.tree) using the the sample X and labels y
    # You will have to make use of the functions in Utility class to train the tree

    # par_node is a parameter that is useful to pass additional information to call 
    # the learn method recursively. Its not mandatory to use this parameter

    # Use the function best_split in Utility class to get the best split and 
    # data corresponding to left and right child nodes

    # One possible way of implementing the tree:
    #    Each node in self.tree could be in the form of a dictionary:
    #       https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
    #    For example, a non-leaf node with two children can have a 'left' key and  a 
    #    'right' key. You can add more keys which might help in classification
    #    (eg. split attribute and split value)
    ### Implement your code here
    #############################################
    max_depth = 10
    if len(y) == 1:   # no data in this group
        return {"leaf": "yes", "label": y}
    elif all(x == y[0] for x in y):   # all y is the same in this group
        return {"leaf": "yes", "label": y[0]}
    elif depth >= max_depth:   # max depth reached 
        return None
    else:
        depth += 1
        split_attribute = best_split(X, y)["split_attribute"]
        split_val = best_split(X, y)["split_val"]

        X_left = best_split(X, y)["X_left"]
        X_right = best_split(X, y)["X_right"]
        y_left = best_split(X, y)["y_left"]
        y_right = best_split(X, y)["y_right"]
        
        root = "{} <= {}".format(split_attribute, split_val)
        tree = {root:[]}
        left = learn(X_left, y_left, {}, depth)
        right = learn(X_right, y_right, {}, depth)
        
        tree[root].append(left)
        tree[root].append(right)
        #root["left"] = learn(X_left, y_left, {}, depth)   
        #root["right"] = learn(X_right, y_right, {}, depth)
        #root["attr"] = split_attribute
        #root["val"] = split_val

        return tree
    #############################################


In [8]:
def classify(tree, record):
    # TODO: classify the record using self.tree and return the predicted label
    ### Implement your code here
    #############################################
    the_tree = tree
    root = list(the_tree.keys())[0]
    attr, comp, val = root.split()
    if record[attr] <= val:
        the_tree = the_tree[root][0]
    else:
        the_tree = the_tree[root][1]
    
    if not isinstance(the_tree, dict):
        return the_tree
    else:
        return classify(record, the_tree)

    #############################################


    record_array = np.array(record)
    labels = []
    the_tree = tree
    
    if type(record[0]) == int or type(record[0]) == float:
        while the_tree.get("val"):
            if record[the_tree["attr"]] <= the_tree["val"]:
                the_tree = the_tree["left"]
            else:
                the_tree = the_tree["right"]
        else:
            labels.append(the_tree.get("label"))    
        #return labels
    else:
        for i, c in enumerate(record_array):
            while the_tree.get("val"):
                if c[the_tree["attr"]] <= the_tree["val"]:
                    the_tree = the_tree["left"]
                else:
                    the_tree = the_tree["right"]
            else:
                labels.append(the_tree.get("label"))    
    return labels


In [9]:
#class RandomForest(object):
num_trees = 10
decision_trees = {}

# the bootstrapping datasets for trees
# bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
bootstraps_datasets = []

# the true class labels, corresponding to records in the bootstrapping datasets
# bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
# the 'i'th bootstrapped dataset.
bootstraps_labels = []

#initiallize
#decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]


In [10]:
def _bootstrapping(XX, n):
    # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
    #
    # TODO: Create a sample dataset of size n by sampling with replacement
    #       from the original dataset XX.
    # Note that you would also need to record the corresponding class labels
    # for the sampled records for training purposes.

    sample = [] # sampled dataset
    labels = []  # class labels for the sampled records
    ### Implement your code here
    #############################################
    XX = np.asarray(XX)
    xx_sample = np.stack(random.choices(XX, k=n))
    xx_sample1 = np.hsplit(xx_sample, len(xx_sample[0]))
    sample = np.concatenate((xx_sample1[0:-1]), axis=1)
    labels = xx_sample1[-1]

    #############################################
    return (sample, labels)

In [11]:
def bootstrapping(XX):
    # Initializing the bootstrap datasets for each tree
    for i in range(num_trees):
        data_sample, data_label = _bootstrapping(XX, len(XX))
        bootstraps_datasets.append(data_sample)
        bootstraps_labels.append(data_label)

In [12]:
def fitting():
    # TODO: Train `num_trees` decision trees using the bootstraps datasets
    # and labels by calling the learn function from your DecisionTree class.
    ### Implement your code here
    #############################################
    for i in range(len(bootstraps_datasets)):
        decision_trees[i]=learn(bootstraps_datasets[i], bootstraps_labels[i], {}, 0)
    #############################################

In [13]:
def voting(X):
    y = []

    for record in X:
        # Following steps have been performed here:
        #   1. Find the set of trees that consider the record as an out-of-bag sample.
        #   2. Predict the label using each of the above found trees.
        #   3. Use majority vote to find the final label for this recod.
        votes = []

        for i in range(len(bootstraps_datasets)):
            dataset = bootstraps_datasets[i]

            if record not in dataset:
                #OOB_tree = decision_trees[i]
                effective_vote = classify(decision_trees[i],record)
                votes.append(effective_vote)
        v = np.array([item for sublist in votes for item in sublist]).astype(int).tolist()
        counts = np.bincount([item for sublist in v for item in sublist])

        if len(counts) == 0:
            # TODO: Special case
            #  Handle the case where the record is not an out-of-bag sample
            #  for any of the trees.
            # NOTE - you can add few lines of codes above (but inside voting) to make this work
            ### Implement your code here
            #############################################
            #d = DecisionTree(max_depth=10)
            vote = classify(tree=decision_trees[1], record=record)
            y = np.append(y, vote)
            #############################################
        else:
            y = np.append(y, np.argmax(counts))

    return y

In [14]:
bootstrapping(XX=XX)

In [15]:
fitting()

KeyboardInterrupt: 

In [ ]:
voting(X=X)